In [7]:
import os

os.chdir("/Users/pascalweiss/dev/python/dl_project_gl_pw/")

In [250]:

from scipy import stats
from src.preprocessing.preprocessing import Preprocessing
from src.preprocessing.datahandler import DataHandler
import torch
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 0
pd.options.display.expand_frame_repr = False


In [10]:

dh = DataHandler()
dh.load_data('data/comments_cleaned.txt', 'data/annotation.txt')
df = dh.get_data_df(deep_copy=False)

In [11]:
pp = Preprocessing(model_type='en')

In [90]:
piped = pp.run_spacy_pipeline(df.reply[:100])
reply = pp.filter_spacy_tokens(piped, no_punctuation=False, no_stop_words=False)
reply = pp.convert_token_docs_text(reply, token_kind='lemma_')
reply = pp.inner_str_join_2d_list(reply)
tfidf = pp.str_list_to_tfidf(reply, min_df=1, ngram_range=(1, 1))
data = tfidf.data

In [308]:
count, x = np.histogram(data, bins=100)
x = x[:len(count)]
idx = np.arange(len(x))

bar_width = 1.0

plt.bar(idx, count, bar_width)
plt.xticks([], [])
plt.show()
print("shape: ", data.shape, stats.describe(data))


In [309]:
v = Variable(torch.FloatTensor(data.reshape(1, len(data))))
data_norm = F.normalize(v)
data_norm_np = data_norm.data.numpy()

count, x = np.histogram(data_norm_np, bins=100)
x = x[:len(count)]
idx = np.arange(len(x))

bar_width = 1.0

plt.bar(idx, count, bar_width)
plt.xticks([], [])
plt.show()
print(stats.describe(data_norm_np.reshape(len(data), )))



In [271]:
batch = Variable(
    torch.FloatTensor([
        (0.1, 0.7, 0.3, 0.9, 0.01, 0.4),
        (10, 21, 4, 0.5, 0.2, 9)
    ])
)

norm1 = F.normalize(batch)
norm2 = F.normalize(batch[:1])
norm3 = F.normalize(batch[1:2])
norm1, norm2, norm3

In [314]:
class DatasetTfIdf(Dataset):
    def __init__(self, tfidf):
        self.tfidf = tfidf

    def __len__(self):
        return self.tfidf.shape[0]

    def __getitem__(self, idx):
        sample = torch.FloatTensor(
            self.tfidf[idx,].todense()
        )

        return sample


ds = DatasetTfIdf(tfidf)
dl = DataLoader(ds, batch_size=2)

## Access all elements WITH normalization and meassure time

In [316]:
%%time
for idx, batch in enumerate(dl):
    # some arbitrary operation
    b = Variable(batch)
    # normalize and do something arbitrary 
    F.normalize(b).sum()

## Access all elements WITHOUT normalization and meassure time

In [317]:
%%time
for idx, batch in enumerate(dl):
    # some arbitrary operation
    b = Variable(batch)
    # normalize and do something arbitrary 
    b.sum()